In [2]:
import numpy as np
import numpy.linalg as LA
import meshplot as mp
from IPython.display import JSON as DJSON
from IPython.display import clear_output
from pspart import Part
from pspart import NNHash
import os
import pandas as ps
from mate_proposals import mate_proposals
from scipy.spatial.transform import Rotation as R
import meshplot as mp
import onshape.brepio as brepio

In [3]:
datapath = '/projects/grail/benjones/cadlab'

In [4]:
loader = brepio.Loader(datapath)

In [5]:
geo, mates = loader.load_flattened('0e2de88aa724d660e4093564_52ea6bfdd317472a99efcffa_316901caae65ff373c71ef88.json')

In [6]:
name = '/fast/jamesn8/assembly_data/assembly_data_with_transforms_all.h5'
assembly_df = ps.read_hdf(name,'assembly')

In [7]:
with open('fully_connected_moving_no_multimates.txt','r') as f:
    set_E_indices = [int(l.rstrip()) for l in f.readlines()]

In [11]:
epsilon_rel = 0.001

for ind in set_E_indices:
    #clear_output(wait=True)
    #display(f'num_processed: {ind}/{assembly_df.shape[0]}')

    #1. spatially hash all MCFs (cache hash maps for each part for re-use with individual mates)
    #2. for all mates, ensure that each MCF is represented (keep track of closest/equivalent MCFs, log percentage of assemblies for which this holds)
    #3. get proposals, edit appropriate ones to true based on equivalence class computed per mated pair of parts (taking outer product of equivalent MCs on left and right)

    assembly_path = assembly_df.loc[ind,'AssemblyPath']
    print('assembly_path:',assembly_path)
    geo, mates = loader.load_flattened(assembly_path + '.json')
    
    parts = [geo[k][1] for k in geo]
    transforms = [geo[k][0] for k in geo]
    mcf_hashes = []
    mc_frames_all = []
    occ_to_index = dict()
    
    #debug
    all_points = []
    
    for j,occ in enumerate(geo):
        occ_to_index[occ] = j
        
    maxdim = max([(part.bounding_box()[1]-part.bounding_box()[0]).max() for part in parts])
    
    for j in range(len(parts)):
        part = parts[j]
        tf = transforms[j]
        mc_frames = []
        for mc in part.all_mate_connectors:
            cs = mc.get_coordinate_system()
            frame = tf[:3,:3] @ cs[:3,:3]
            origin = tf[:3,:3] @ cs[:3,3] + tf[:3,3]
            all_points.append(origin)
            rot = R.from_matrix(frame).as_quat()
            mc_frames.append(np.concatenate([origin, rot]))
        mc_frames_all.append(mc_frames)
        frame_hash = NNHash(mc_frames, 7, maxdim * epsilon_rel)
        #frame_hash = NNHash([mc_frame[:3] for mc_frame in mc_frames], 3, maxdim * epsilon_rel)
        mcf_hashes.append(frame_hash)
    
    all_points = np.array(all_points)
    #p = mp.plot(all_points)
    print(f'threshold: {maxdim * epsilon_rel}')
    num_invalid_mates = 0
    for mate in mates:
        for i in range(2):
            occId = mate.matedEntities[i][0]
            partIndex = occ_to_index[occId]
            assert(list(geo.keys())[partIndex] == occId)
            origin_local = mate.matedEntities[i][1][0]
            frame_local = mate.matedEntities[i][1][1]
            tf = transforms[partIndex]
            origin = tf[:3,:3] @ origin_local + tf[:3,3]            
            frame = tf[:3,:3] @ frame_local
            #p.add_points(origin[np.newaxis],shading={'point_color':'blue','point_size':0.01})
            rot = R.from_matrix(frame).as_quat()
            mc_frame = np.concatenate([origin, rot])
            found=False
            minDist = np.inf
            for k in range(len(mc_frames_all[partIndex])):
                dist = LA.norm(mc_frames_all[partIndex][k] - mc_frame)
                if dist < minDist:
                    minDist = dist
                if dist < maxdim * epsilon_rel:
                    found=True
                    break
            
            #neighbors = mcf_hashes[occ_to_index[occId]].get_nearest_points(np.concatenate([origin, rot]))
            #neighbors = mcf_hashes[occ_to_index[occId]].get_nearest_points(origin)
            print(f'{i} closest dist: {minDist}')
            if not found:
                num_invalid_mates += 1
                print(f'invalid mate {mate.name}') 
                break
    print(f'num invalid mates: {num_invalid_mates}/{len(mates)}')
    break

assembly_path: 0e2de88aa724d660e4093564_52ea6bfdd317472a99efcffa_316901caae65ff373c71ef88
threshold: 0.00027400000000000005
0 closest dist: 0.0
1 closest dist: 0.0
0 closest dist: 2.5713883747773856e-17
1 closest dist: 1.0408340855860843e-17
0 closest dist: 1.3877787807814457e-17
1 closest dist: 0.002700000000000001
invalid mate Revolute Servo-Servo axle
0 closest dist: 3.2526065174565133e-19
1 closest dist: 0.0067178457518662
invalid mate Fastened Servo-Base
0 closest dist: 0.0
1 closest dist: 0.009552486587271397
invalid mate Fastened Antenna-Base
0 closest dist: 6.938893903907228e-18
1 closest dist: 0.3978302860652944
invalid mate Revolute Bearing F
0 closest dist: 1.3877787807814457e-17
1 closest dist: 0.1720758226916534
invalid mate Revolute Bearing B
0 closest dist: 1.3877787807814457e-17
1 closest dist: 0.1057428846254173
invalid mate Revolute Bearing R
0 closest dist: 6.938893903907228e-18
1 closest dist: 1.1136500420051743
invalid mate Revolute Bearing L
0 closest dist: 3.1031

In [ ]:
list(mate_subset['MateIndex'])

[376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392]